In [ ]:
import numpy as np
import pandas as pd
import os
from statsmodels.tsa.arima_process import arma_generate_sample

from IPython.display import clear_output

# Seteamos pandas para que muestre todas las columnas y más filas
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# np.random.seed(12361)  # Semilla 2023
# np.random.seed(345827) # Semilla 2024

In [ ]:
PATH = os.path.join(os.getcwd(), 'databases')

if not os.path.exists(PATH):
    os.makedirs(PATH)

## **Configuración de los parámetros de las simulaciones**
---
* `n_simulations`: Número de simulaciones
* `n_total`: Número de elementos en la muestra (controles, tratados y Ni Ni)
* `control_pctg`: Porcentaje de individuos que van a ser tratados
* `n_control`, `n_treated`: Representan las cantidades de tratados y controles
* `n`: Número de elementos que son o bien tratado o controles. Se calcula como `n_control` + `n_treated`
* `T`: Número de períodos que son observados los individuos
* `first_tr_period`: Primer período de tratamiento (o único dependiendo de la cantiadad de cohortes=n_cohorts)

* `phiNini`: Persistencia auto-regresiva asociada a la variable resultado de los Nini
* `phiT`: Persistencia auto-regresiva asociada a la variable resultado de los tratados
* `phiC`: Persistencia auto-regresiva asociada a la variable resultado de los controles
* `meanFEN`: Media (nivel) del efecto fijo de los Nini
* `meanFET`: Media (nivel) del efecto fijo de los tratados
* `meanFEC`: Media (nivel) del efecto fijo de los controles
* `NivelN`: Media (nivel) de la variable resultado de los Nini sin el fijo
* `NivelT`: Media (nivel) de la variable resultado de los tratados sin el fijo
* `NivelC`: Media (nivel) de la variable resultado de los controles sin el fijo
* `ImpactoPrporcional`: Porcentaje promedio del impacto tipo escalón sobre la la media del nivel de la variable resultado
* `STDImpacto`: Desviación estándar de la distribución del impacto.
* `Dependencia`: Si se simular dependencia en la entrada al programa, si es 1 se genera con dependencia el ingreso al tratamiento.
* `hetecohorte`: Heterogeneidad en el impacto por cohorte. Cambia la estructura dinámica del escalón. Puede ser aleatorio o mediante un proceso ARMA (caso donde la variable hetecohorte = 1), en este último caso, la dinámica del modelo ARMA cambia para cada cohorte.
* `NPerDep`: Número de períodos de dependencia para la participación en el tratamiento, pueden ser 4 ó 7.
* `nolineal`: Es 1 (uno) si se quiere simular una dependencia de entrada al tratamiento que sea no lineal, es 0 (cero) si la dependencia es lineal
* `ini`: cantidad de observaciones que se queman para que la dinámica del proceso auto-regresivo de la variable resultado esté ya en situación de estado estacionario
* `n_cohorts`: Número de cohortes
* `StdErrorSerie`: Desviación estándar del térmio de ruido de las series

In [ ]:
n_simulations = 1

n_total = 5000
treated_pctg = 0.10
control_pctg = 0.10

n_treated = round(treated_pctg * n_total)
n_control = n_treated
n = n_treated + n_control
n_nini = n_total - n

T = 20
ini = 200
total_periods = T + ini
first_tr_period = 12

phiNini = 0.90
phiT = 0.90
PhiC = 0.90
MeanFEN = 10.0
MeanFET = 10.0
MeanFEC = 10.0

NivelN = MeanFEN
NivelT = MeanFET
NivelC = MeanFEC
ImpactoPrporcional = 0.05
ImpactoNominal = NivelC * ImpactoPrporcional
STDImpacto = 0.05

Dependencia = 1

porcentaje  = 0.05
hetecohorte = 1
NPerDep = 3
nolineal = 0

n_cohorts = 3
StdErrorSerie = 5

In [ ]:
# Variable evolution matrix for control and treated
YPanel = np.zeros(shape=(n_simulations*n*T,9))

# Variable evolution matrix for nini
YPanelNiNi = np.zeros(shape=(n_simulations*n_nini*T,9))

In [ ]:
def process_entry(y, t, n_dep_periods, non_linear, pctg, dependence):
    """
    Evaluates if the time series is increasing in the last n_dep_periods periods
    starting from the period t and optionally if the series is increasing in the
    last two periods with a percentage greater than pctg.

    Args:
        y (array): time series
        t (int): current time index
        n_dep_periods (int): amount of periods to consider for the temporal dependency
        non_linear (bool): indicates whether to check for an extra condition
        pctg (float): _description_
        dependence (bool): indicates whether to check for the increasing trend
        in the series

    Returns:
        bool: True if the condition is met, False otherwise
    """
    if dependence == 1:
        start = t - n_dep_periods
        condition = all(
            y[start+i] < y[start+i+1] for i in range(n_dep_periods-1)
        )
        if non_linear:
            condition &= (abs(y[t-2] - y[t-1]) / y[t-2]) > pctg
    else:
        condition = True
    return condition

In [ ]:
for sim in range(n_simulations):
    EfectoFijoT = np.random.normal(0,1,n) + MeanFET
    EfectoTemporalT = np.random.normal(0,1,T+ini)

    Y = np.zeros(shape=(n,T))   # n = n_treated + n_control
    y_control = np.zeros(shape=(n_control, T))
    y_counterfac = np.zeros(shape=(n, T))
    treatment_starts = []

    n_treated_in_cohort = int(n_treated / n_cohorts) # Amount of treated units per cohort
    i = 0

    for dataset, label in [(Y, 'Tratados'), (y_control, 'Controles')]:
        for cohort in range(n_cohorts):
            print(f"Cohorte {cohort+1} de {n_cohorts}")
            i = cohort * n_treated_in_cohort
            ii = 0
            while i <= ((cohort + 1) * n_treated_in_cohort - 1):
                y = np.zeros(total_periods)
                y[0] = (
                    MeanFET + EfectoFijoT[i] + EfectoTemporalT.mean()
                    + np.random.normal(0, 1) * StdErrorSerie
                )
                Tr, use = 0, False
                treatment_start = ini + first_tr_period + cohort
                for t in range(total_periods):
                    if (t == treatment_start) and (Tr == 0):
                        use = process_entry(y, t, NPerDep, nolineal, porcentaje, Dependencia)
                        if use:
                            Tr = t - ini
                    y[t] = (
                        (1 - phiT) * (MeanFET + EfectoFijoT[i]
                        + EfectoTemporalT.mean()) + phiT * y[t - 1]
                        + np.random.normal(0, 1) * StdErrorSerie
                    )
                if use:
                    if ii % 100 == 0:
                        print(
                            f"{label}: entrada en {treatment_start-ini} {ii} de {n_treated_in_cohort} "
                            f"Simulación {sim} de {n_simulations}"
                        )
                    treatment_starts.append(treatment_start-ini)
                    dataset[i, :] = y[ini:]
                    i += 1
                    ii += 1

    clear_output(wait=True)
    Y[n_treated:,] = y_control
    treatment_starts += treatment_starts
    y_counterfac[:n_treated,] = Y[:n_treated,]

    # INCORPORACIÓN DEL TRATAMIENTO
    for i in range(n_treated):
        iniTr = treatment_starts[i]
        if hetecohorte == 1:
            phi_Tra = [0.9, 0.8, 0.7, 0.6, 0.5, 0.6, 0.4]
            phi_trat_par = phi_Tra[iniTr - 5]
            arparams = np.array([phi_trat_par, 0])
            maparams = np.array([0, 0])
            arparams = np.r_[1, -arparams]
            maparams = np.r_[1, maparams]
            impacto = arma_generate_sample(
                arparams, maparams, (T-iniTr+1), burnin=5000
            ) + ImpactoNominal
            Y[i,(iniTr-1):] += impacto
        else:
            TraCondicion = np.array(treatment_starts)
            Y[i,(iniTr-1):] += np.random.normal(ImpactoNominal,STDImpacto,(T-iniTr+1))
            TraCondicion = np.array(treatment_starts)

In [ ]:
    # INCORPORACIÓN DEL TRATAMIENTO
    for i in range(n_treated):
        iniTr = TraInicio[i]
        if hetecohorte == 1:
            phi_Tra = [0.9, 0.8, 0.7, 0.6, 0.5, 0.6, 0.4]
            phi_trat_par = phi_Tra[iniTr - 5]
            arparams = np.array([phi_trat_par, 0])
            maparams = np.array([0, 0])
            arparams = np.r_[1, -arparams]
            maparams = np.r_[1, maparams]
            impacto = arma_generate_sample(
                arparams, maparams, (T-iniTr+1), burnin = 5000
            ) + ImpactoNominal
            Y[i,(iniTr-1):] += impacto
        else:
            TraCondicion = np.array(TraInicio)
            Y[i,(iniTr-1):] += np.random.normal(ImpactoNominal,STDImpacto,(T-iniTr+1))
            TraCondicion = np.array(TraInicio)

    ycte = np.zeros(shape=(n*T,1))
    ycte_Cfactual = np.zeros(shape=(n*T,1))
    tiempo = np.zeros(shape=(n*T,1))
    individuo = np.zeros(shape=(n*T,1))
    IniTratmiento = np.zeros(shape=(n*T,1))
    tratado = np.zeros(shape=(n*T,1))
    control = np.zeros(shape=(n*T,1))
    Nini = np.zeros(shape=(n*T,1))
    for i in range(n):
        iniTr = TraInicio[i]
        cte1 = Y[i,:]
        ycte[(i*T):((i+1)*T)] = np.reshape(cte1,(T,1))
        cte1_Cfactual = YcontraFac[i,:]
        ycte_Cfactual[(i*T):((i+1)*T)] = np.reshape(cte1_Cfactual,(T,1))
        cte2 = list(range(1,T+1))
        tiempo[(i*T):((i+1)*T)] = np.reshape(cte2, (T,1))
        individuo[(i*T):((i+1)*T)] = i
        IniTratmiento[(i*T):((i+1)*T)] = iniTr
        if i<n_treated:
            tratado[(i*T):((i+1)*T)] = 1
        else:
            control[(i*T):((i+1)*T)] = 1

    YPanel[(sim*(n*T)):((n*T)*(sim+1)),0] = sim+1
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),1] = np.reshape(individuo, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),2] = np.reshape(tiempo, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),3] = np.reshape(ycte, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),4] = np.reshape(IniTratmiento, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),5] = np.reshape(tratado, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),6] = np.reshape(control, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),7] = np.reshape(Nini, (n*T,))
    YPanel[(sim*(n*T)):((n*T)*(sim+1)),8] = np.reshape(ycte_Cfactual, (n*T,))

    YN = np.zeros(shape=(n_nini,T))
    EfectoFijoT = np.random.normal(0,1,n_nini)+MeanFEN

    ycte = np.zeros(shape=(n_nini*T,1))
    tiempo = np.zeros(shape=(n_nini*T,1))
    individuo = np.zeros(shape=(n_nini*T,1))
    IniTratmiento = np.zeros(shape=(n_nini*T,1))
    tratado = np.zeros(shape=(n_nini*T,1))
    control = np.zeros(shape=(n_nini*T,1))
    Nini = np.zeros(shape=(n_nini*T,1))
    for i in range(n_nini):
        y = np.zeros(total_periods)
        y[0] = MeanFEN + EfectoFijoT[i] + EfectoTemporalT.mean() + np.random.normal(0,1) * StdErrorSerie
        for t in total_periods:
            y[t] = (1-phiNini) * (MeanFEN + EfectoFijoT[i] + EfectoTemporalT.mean()) + phiNini * y[t - 1] + np.random.normal(0, 1) * StdErrorSerie
        YN[i,:] = y[ini:]
        cte1 = YN[i,:] + 0
        ycte[(i*T):((i+1)*T)] = np.reshape(cte1,(T,1))
        cte2 = list(range(1,T+1))
        tiempo[(i*T):((i+1)*T)] = np.reshape(cte2, (T,1))
        individuo[(i*T):((i+1)*T)] = i + n
        Nini[(i*T):((i+1)*T)] = 1

    YPanelNiNi[(sim*(n_nini*T)):((n_nini*T)*(sim+1)),0] = sim+1
    YPanelNiNi[(sim*(n_nini*T)):((n_nini*T)*(sim+1)),1] = np.reshape(individuo, (n_nini*T,))
    YPanelNiNi[(sim*(n_nini*T)):((n_nini*T)*(sim+1)),2] = np.reshape(tiempo, (n_nini*T,))
    YPanelNiNi[(sim*(n_nini*T)):((n_nini*T)*(sim+1)),3] = np.reshape(ycte, (n_nini*T,))
    YPanelNiNi[(sim*(n_nini*T)):((n_nini*T)*(sim+1)),7] = np.reshape(Nini, (n_nini*T,))